## Exercise 1
#### Multinomial distributed random numbers

##### a) Create a multinomial distribution from the example file anthrokids.csv: Use the column age and round the floating point values to obtain integers. Count the frequencies of the integers and use them as basis of the distribution.

In [1]:
import pandas as pd
import numpy as np

#Read the file into a Pandas DataFrame
raw_data = pd.read_csv("anthrokids.csv")
raw_data

,id,mass,height,waist,foot,sittingHeight,upperLegLength,kneeHeight,forearmLength,age,gender,handedness,birthOrder
0,1,15.5,103.3,47.5,16.3,582.0,306.0,NaN,259.0,4.219,F,right,1.0
1,2,17.6,103.9,49.8,16.3,606.0,311.0,NaN,274.0,4.326,M,right,1.0
2,3,23.0,111.2,52.0,17.1,594.0,387.0,NaN,304.0,4.476,F,right,1.0
3,4,16.5,99.7,49.1,16.3,542.0,312.0,NaN,281.0,3.841,F,both,1.0
4,5,15.0,99.7,46.5,16.7,524.0,321.0,NaN,269.0,3.460,F,both,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,91096,16.0,103.6,48.7,16.5,597.0,307.0,307.0,269.0,3.403,M,both,2.0
3896,91097,14.3,99.9,46.0,15.1,582.0,291.0,275.0,253.0,3.397,M,left,4.0
3897,91099,12.4,90.5,47.6,14.1,537.0,289.0,254.0,224.0,2.442,F,right,1.0
3898,91100,18.4,100.7,55.1,16.1,570.0,320.0,303.0,273.0,4.188,M,right,1.0


In [2]:
#Replacing null values with zeroes
raw_data['age'] = raw_data['age'].fillna(0)

# Extract the "age" column and round the floating point values to obtain integers.
age_col_only = raw_data["age"]
age_col_only = age_col_only.round().astype('int')

# Count the frequencies of the integers - get uniques age values and count the frequencies.
unique_age = np.unique(age_col_only)
frequencies = [np.count_nonzero(age_col_only == age) for age in unique_age]

# Calculate the probabilities of each integer value
total_observations = len(age_col_only)
probabilities = [freq/total_observations for freq in frequencies]

print('Probabilities:', probabilities)

Probabilities: [0.0005128205128205128, 0.014358974358974359, 0.040256410256410254, 0.05846153846153846, 0.06948717948717949, 0.06282051282051282, 0.05923076923076923, 0.05076923076923077, 0.0658974358974359, 0.06615384615384616, 0.07230769230769231, 0.07358974358974359, 0.08076923076923077, 0.06948717948717949, 0.06769230769230769, 0.05076923076923077, 0.056666666666666664, 0.03564102564102564, 0.004871794871794872, 0.0002564102564102564]


The below loop that creates a random number between 0 and 1 and then determines which integer value corresponds to that probability is used to create the multinomial distribution. 

Here we used a cumulative probability method in which the probabilities of each integer value were added up until the random number was reached. The multinomial dist array is then updated with the integer value representing the probability. For all the observations, we go through this process again.

In [3]:
# Use the probabilities to generate a multinomial distribution
mult_dist = np.zeros(len(probabilities))
for i in range(total_observations):
    random_numbers = np.random.uniform()
    cum_prob = 0
    for j in range(len(probabilities)):
        cum_prob += probabilities[j]
        if random_numbers <= cum_prob:
            mult_dist[j] += 1
            break

print("Multinomial distribution:", mult_dist)

Multinomial distribution: [  5.  64. 151. 242. 257. 224. 227. 212. 267. 263. 298. 297. 310. 271.
 285. 194. 204. 112.  15.   2.]



##### b) Sample from the multinomial distribution.


In [4]:
# Sample from the multinomial distribution
n_samples = 100

# selects a random sample of specified size from the unique_ages list, with the probability of each age being selected determined by the probabilities list.
samples = np.random.choice(unique_age, size=n_samples, p=probabilities)

# Print the samples
print("Multinomial samples:", samples)

Multinomial samples: [ 6  7 16 15 10  8  7 13 10  8  5 11 12 10  9  4  8 12 18 12  6 16  9 10
 14  6 12  9 14 13  7  3  8 14  6 18 13  9 10 16 13 18  9  7  5  4 10  3
  7  6  4 11  4  3 18  3 10  8 18 12 14  6  3  9  6  4  5  9 12  6  3  7
  8 12 14 10 12  8 12  5 16 14 12 13  9  4  4  3 15  4 12 11  6 12 14 13
  8 16 11 10]


##### c) How can you check if your samples really follow the distribution from 1 a)?


A Pearson’s chi-square test is a Hypothesis testing. It is used to determine whether your data are significantly different from what you expected.

In [5]:
from scipy.stats import chisquare

# Define the expected counts based on the probabilities
num_trials = 1000
expected_counts = [prob * num_trials for prob in probabilities]

# Generate the multinomial sample
multinomial_sample = np.random.multinomial(num_trials, probabilities)

# Perform the Chi-Square Goodness of Fit test
test_statistic, p_value = chisquare(multinomial_sample, expected_counts)

print("Chi-Square test statistic:", test_statistic)
print("p-value:", p_value)


Chi-Square test statistic: 22.5387996238063
p-value: 0.2582668487232401


As the p-value in this case is higher than the required significance value (0.05), the multinomial sample is not substantially different from the predicted counts and so follows the multinomial distribution. If not, we reject the null hypothesis that the sample exhibits the predicted distribution.